In [3]:
import itertools
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor'] = 'w'
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['legend.framealpha'] = 1
import numpy as np
import progressbar as pb
import scipy.special as ss

In [4]:
max_p = 5
max_t = 5

p_0 = list(itertools.product(*[[0,], np.arange(1, max_t+1)]))
p_greater = list(itertools.product(*[np.arange(1, max_p+1), [0,]]))
p_and_t_list = p_0 + p_greater
slot_list = list(itertools.product(*[p_and_t_list, np.arange(33)]))

In [5]:
attack_length = 2
length_n_lists = list(itertools.product(*[slot_list]*attack_length))

In [6]:
len(length_n_lists)

108900

In [7]:
330**2

108900

In [109]:
def calcSelfishDelay(p_list, e_list):
    return 60 * attack_length + 40 * sum(p_list) + 8 * sum(map(lambda e: max(24 - e, 0), e_list[1:]))

def calcHonestDelay(e_list, t_list):
    return 60 * attack_length + 40 * sum(t_list) + 8 * sum(map(lambda e: max(e - 8, 0), e_list))

def calcDelayDiff(p_list, e_list, t_list):
    return calcSelfishDelay(p_list, e_list) - calcHonestDelay(e_list, t_list)

In [110]:
def calcProb(alpha, p_list, e_list, t_list):
    term1 = np.product(list(map(lambda e : ss.binom(32, e), e_list)))
    term2 = np.power(1-alpha, 33*attack_length + sum(p_list) - sum(e_list))
    term3 = np.power(alpha, attack_length + sum(t_list) + sum(e_list))
    return term1 * term2 * term3

In [111]:
def getLists(slots):
    p_list = []
    t_list = []
    e_list = []
    for slot in slots:
        p_list.append(slot[0][0])
        t_list.append(slot[0][1])
        e_list.append(slot[1])
    return p_list, t_list, e_list

In [112]:
alpha = 0.4

probabilites = []
bar = pb.ProgressBar()
for slots in bar(length_n_lists):
    p_list, t_list, e_list = getLists(slots)
    if (calcDelayDiff(p_list, e_list, t_list) < 0):
        probabilites.append(calcProb(alpha, p_list, e_list, t_list))

100% (35937000 of 35937000) |############| Elapsed Time: 0:36:22 Time:  0:36:22


In [113]:
sum(probabilites), sum(probabilites) * 365 * 24 * 60

(0.01621064031969418, 8520.31255203126)

In [114]:
len(probabilites)

21112847